# this notebook extracts a lot of data from plate models and plots them

In [3]:
import pygplates
import points_in_polygons
import numpy as np
import pandas as pd
import pickle
import os
from mpl_toolkits.basemap import Basemap, shiftgrid
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
from create_gpml import create_gpml_regular_long_lat_mesh
from skimage import measure
%matplotlib inline

In [4]:
#load files

basedir = '/Users/Andrew/Documents/PhD/Models'  #set basedir for load
basedir_save = '/Users/Andrew/Documents/PhD/Scripts/Scripts_Output/Python/2017_Rift_SZ_length/' #set basedir for saving

#load COB files
COBterrane_file_410_0 = '%s/Matthews-410-0Ma/StaticGeometries/AgeGridInput/Global_EarthByte_GeeK07_COB_Terranes_Matthews_etal_EDITED.gpml' % basedir
COBterrane_file_1000_500 = '%s/1000-410_Models/COBs_global_artificial_individual.gpml' % basedir
COBterrane_file_500_410 = '/Users/Andrew/Documents/PhD/Models/Domeier_models/Merge/Matthews_COBs_for_Dom-models.gpml'

#load polygon files
craton_file_1000_500 = '%s/1000-410_Models/Topos_for_plotting/Neoproterozoic_Palaeozoic_Shapes_20170714_new-antarctica.gpml' % basedir
craton_file_500_410 = '/Users/Andrew/Documents/PhD/Models/Domeier_models/Merge/Domeier-EarlyP-merge-Land_ASM.gpml'
craton_file_410_0 = '%s/Matthews-410-0Ma/StaticGeometries/Coastlines/Global_coastlines_2015_v1_low_res.gpml' % basedir

#load rotation files
rotation_file_410_250 = '/Applications/GPlates-2.0.0_r18604M/SampleData/FeatureCollections/Rotations/Matthews_etal_GPC_2016_410-0Ma_GK07.rot'
rotation_file_250_0 = '%s/Matthews-410-0Ma/Global_EB_250-0Ma_GK07_Matthews_etal.rot' % basedir
rotation_file_1000_500 = '%s/1000-410_Models/Topos_for_plotting/1000-410_rotations(finished)_20170621a_north_china-tarim-aus.rot' % basedir
rotation_file_500_410 = '/Users/Andrew/Documents/PhD/Models/Domeier_models/Merge/Domeier-EarlyP-merge-Rotation_ASM.rot'

#generate pygplates files for Mer17 models
cobter_1000_500 = pygplates.FeatureCollection(COBterrane_file_1000_500)
rotation_model_1000_500 = pygplates.RotationModel(rotation_file_1000_500)
cratons_1000_500 = pygplates.FeatureCollection(craton_file_1000_500)

#generate pygplates files for Dom1618 models
cobter_500_410 = pygplates.FeatureCollection(COBterrane_file_500_410)
rotation_model_500_410 = pygplates.RotationModel(rotation_file_500_410)
cratons_500_410 = pygplates.FeatureCollection(craton_file_500_410)


#generate pygpltaes file for Mat16 models
cobter_410_0 = pygplates.FeatureCollection(COBterrane_file_410_0)
rotation_model_410_250 = pygplates.RotationModel(rotation_file_410_250)
rotation_model_250_0 = pygplates.RotationModel(rotation_file_250_0)
cratons_410_0 = pygplates.FeatureCollection(craton_file_410_0)

#generate points for creating supercontinent polygons
multipoints = create_gpml_regular_long_lat_mesh(1)

for multipoint in multipoints:
    for mp in multipoint.get_all_geometries():
        points = mp.to_lat_lon_point_list()

#set date line wrapper for plotting maps
date_line_wrapper_crat = pygplates.DateLineWrapper()

In [5]:
#create supercotinent polygons
polygons = []
for feature in cobter_410_0:
    
    for geom in feature.get_geometries():
        polygon = []
        #print feature.get_geometry()
    
        polygon = feature
        polygon.set_geometry(pygplates.PolygonOnSphere(geom))

        #print polygon.get_geometry()

        polygons.append(polygon)

cobter_410_0 = pygplates.FeatureCollection(polygons)
#fc.write('pltest.gpmlz')
#
polygons = []
for feature in cobter_1000_500:

    for geom in feature.get_geometries():
        polygon = []
        #print feature.get_geometry()
    
        polygon = feature
        polygon.set_geometry(pygplates.PolygonOnSphere(geom))

        #print polygon.get_geometry()

        polygons.append(polygon)

cobter_1000_500 = pygplates.FeatureCollection(polygons)
#fc.write('pltest.gpmlz')

polygons = []
for feature in cobter_500_410:
    #print feature
    for geom in feature.get_geometries():
        polygon = []
        #print feature.get_name()
    
        polygon = feature
        polygon.set_geometry(pygplates.PolygonOnSphere(geom))

        #print polygon.get_geometry()

        polygons.append(polygon)

cobter_500_410 = pygplates.FeatureCollection(polygons)
#fc.write('pltest.gpmlz')

In [6]:
#this function creates new polygons that are the amalgamation of a series of overlapping polygons
#in order to make 'supercontinent polyogns'
def run_grid_pip(time,points,polygons,rotation_model):
    
    reconstructed_polygons = []
    pygplates.reconstruct(polygons,rotation_model,reconstructed_polygons,time)
    
    rpolygons = []
    for polygon in reconstructed_polygons:
        #print polygon.get_description()
        #if polygon.get_reconstructed_geometry() == 'PolygonOnSphere':
        if polygon.get_reconstructed_geometry():
            #pri#nt polygon.get_reconstructed_geometry()
            rpolygons.append(polygon.get_reconstructed_geometry())


    polygons_containing_points = points_in_polygons.find_polygons(points, rpolygons)

    lat = []
    lon = []
    zval = []
    
    for pcp,point in zip(polygons_containing_points,points):
        lat.append(point.get_latitude())
        lon.append(point.get_longitude())
        if pcp is not None:
            zval.append(1)
        else:
            zval.append(0)
            
    bi = np.array(zval).reshape(181,361)
    
    return bi

## Extract area of and number of continents from model

In [ ]:
#extract data for 1000-510
date_line_wrapper = pygplates.DateLineWrapper()
plate_area_neop = [] 
plate_length_neop = []
plate_area_neop2 = [] 
plate_length_neop2 = []
for i,time in enumerate(np.arange(510,1010,10)):
    print time
    
    bi = run_grid_pip(time,points,cobter_1000_500,rotation_model_1000_500)
    
    contours = measure.find_contours(bi, .5)
    
    pad_hor = np.zeros((1,361))
    pad_ver = np.zeros((182,1))
    pad1 = np.vstack((bi,pad_hor))
    pad2 = np.hstack((pad_ver,pad1))
    pad3 = np.hstack((pad2,pad_ver))
    contours = measure.find_contours(pad3, 0.5, fully_connected='low')

    contour_polygons = []
    cobter_feature_collection = []
    for n,cp in enumerate(contours):
        cp[:,1] = cp[:,1]-1
        cp[:,0] = cp[:,0]-1

        cp[np.where(cp[:,0]<0.),0] = 0
        cp[np.where(cp[:,0]>180.),0] = 180
        cp[np.where(cp[:,1]<0.),1] = 0
        cp[np.where(cp[:,1]>360.),1] = 360
        cpf = pygplates.PolygonOnSphere(zip(cp[:,0]-90,cp[:,1]-180))
        contour_polygons.append(cpf)
        cobter_feature_collection.append(cpf)
        
    area_threshold = 500000
    area_threshold2 = 10000

    tmp_neop_area = []
    tmp_neop_len = []
    tmp_neop_area2 = []
    tmp_neop_len2 = []
    for pg in contour_polygons:
        if pg.get_area()*pygplates.Earth.mean_radius_in_kms**2 > area_threshold:
            tmp_neop_area.append(pg.get_area()*pygplates.Earth.mean_radius_in_kms**2)
            tmp_neop_len.append(pg.get_arc_length()*pygplates.Earth.mean_radius_in_kms)
        if pg.get_area()*pygplates.Earth.mean_radius_in_kms**2 > area_threshold2:
            tmp_neop_area2.append(pg.get_area()*pygplates.Earth.mean_radius_in_kms**2)
            tmp_neop_len2.append(pg.get_arc_length()*pygplates.Earth.mean_radius_in_kms)

    plate_area_neop.append(tmp_neop_area)
    plate_length_neop.append(tmp_neop_len)
    plate_area_neop2.append(tmp_neop_area2)
    plate_length_neop2.append(tmp_neop_len2)

In [ ]:
#extract data for 500-410
date_line_wrapper = pygplates.DateLineWrapper()
plate_area_DomPalaeo = [] 
plate_length_DomPalaeo = []
plate_area_DomPalaeo2 = [] 
plate_length_DomPalaeo2 = []
for i,time in enumerate(np.arange(420,510,10)):
    print time
    
    bi = run_grid_pip(time,points,cobter_500_410,rotation_model_500_410)
    
    contours = measure.find_contours(bi, .5)
    
    pad_hor = np.zeros((1,361))
    pad_ver = np.zeros((182,1))
    pad1 = np.vstack((bi,pad_hor))
    pad2 = np.hstack((pad_ver,pad1))
    pad3 = np.hstack((pad2,pad_ver))
    contours = measure.find_contours(pad3, 0.5, fully_connected='low')

    contour_polygons = []
    cobter_feature_collection = []
    for n,cp in enumerate(contours):
        cp[:,1] = cp[:,1]-1
        cp[:,0] = cp[:,0]-1

        cp[np.where(cp[:,0]<0.),0] = 0
        cp[np.where(cp[:,0]>180.),0] = 180
        cp[np.where(cp[:,1]<0.),1] = 0
        cp[np.where(cp[:,1]>360.),1] = 360
        cpf = pygplates.PolygonOnSphere(zip(cp[:,0]-90,cp[:,1]-180))
        contour_polygons.append(cpf)
        cobter_feature_collection.append(cpf)
        
    area_threshold = 500000
    area_threshold2 = 10000

    tmp_palaeo_area = []
    tmp_palaeo_len = []
    tmp_palaeo_area2 = []
    tmp_palaeo_len2 = []
    for pg in contour_polygons:
        if pg.get_area()*pygplates.Earth.mean_radius_in_kms**2 > area_threshold:
            tmp_palaeo_area.append(pg.get_area()*pygplates.Earth.mean_radius_in_kms**2)
            tmp_palaeo_len.append(pg.get_arc_length()*pygplates.Earth.mean_radius_in_kms)
        if pg.get_area()*pygplates.Earth.mean_radius_in_kms**2 > area_threshold2:
            tmp_palaeo_area2.append(pg.get_area()*pygplates.Earth.mean_radius_in_kms**2)
            tmp_palaeo_len2.append(pg.get_arc_length()*pygplates.Earth.mean_radius_in_kms)

    plate_area_DomPalaeo.append(tmp_palaeo_area)
    plate_length_DomPalaeo.append(tmp_palaeo_len)
    plate_area_DomPalaeo2.append(tmp_palaeo_area2)
    plate_length_DomPalaeo2.append(tmp_palaeo_len2)

In [ ]:
#extract data for 260-410
date_line_wrapper = pygplates.DateLineWrapper()
plate_area_palaeo = [] 
plate_length_palaeo = []
plate_area_palaeo2 = [] 
plate_length_palaeo2 = []
for i,time in enumerate(np.arange(260,420,10)):
    print time
    
    bi = run_grid_pip(time,points,cobter_410_0,rotation_model_410_250)
    
    contours = measure.find_contours(bi, .5)
    
    pad_hor = np.zeros((1,361))
    pad_ver = np.zeros((182,1))
    pad1 = np.vstack((bi,pad_hor))
    pad2 = np.hstack((pad_ver,pad1))
    pad3 = np.hstack((pad2,pad_ver))
    contours = measure.find_contours(pad3, 0.5, fully_connected='low')

    contour_polygons = []
    cobter_feature_collection = []
    for n,cp in enumerate(contours):
        cp[:,1] = cp[:,1]-1
        cp[:,0] = cp[:,0]-1

        cp[np.where(cp[:,0]<0.),0] = 0
        cp[np.where(cp[:,0]>180.),0] = 180
        cp[np.where(cp[:,1]<0.),1] = 0
        cp[np.where(cp[:,1]>360.),1] = 360
        cpf = pygplates.PolygonOnSphere(zip(cp[:,0]-90,cp[:,1]-180))
        contour_polygons.append(cpf)
        cobter_feature_collection.append(cpf)
        
    area_threshold = 500000
    area_threshold2 = 10000

    tmp_palaeo_area = []
    tmp_palaeo_len = []
    tmp_palaeo_area2 = []
    tmp_palaeo_len2 = []
    for pg in contour_polygons:
        if pg.get_area()*pygplates.Earth.mean_radius_in_kms**2 > area_threshold:
            tmp_palaeo_area.append(pg.get_area()*pygplates.Earth.mean_radius_in_kms**2)
            tmp_palaeo_len.append(pg.get_arc_length()*pygplates.Earth.mean_radius_in_kms)
        if pg.get_area()*pygplates.Earth.mean_radius_in_kms**2 > area_threshold2:
            tmp_palaeo_area2.append(pg.get_area()*pygplates.Earth.mean_radius_in_kms**2)
            tmp_palaeo_len2.append(pg.get_arc_length()*pygplates.Earth.mean_radius_in_kms)

    plate_area_palaeo.append(tmp_palaeo_area)
    plate_length_palaeo.append(tmp_palaeo_len)
    plate_area_palaeo2.append(tmp_palaeo_area2)
    plate_length_palaeo2.append(tmp_palaeo_len2)

In [ ]:
#extract data for 260-0
date_line_wrapper = pygplates.DateLineWrapper()
plate_area_mesoceno = [] 
plate_length_mesoceno = []

plate_area_mesoceno2 = [] 
plate_length_mesoceno2 = []

for i,time in enumerate(np.arange(0,260,10)):
    print time
      
    bi = run_grid_pip(time,points,cobter_410_0,rotation_model_250_0)
    
    contours = measure.find_contours(bi, .5)
    
    pad_hor = np.zeros((1,361))
    pad_ver = np.zeros((182,1))
    pad1 = np.vstack((bi,pad_hor))
    pad2 = np.hstack((pad_ver,pad1))
    pad3 = np.hstack((pad2,pad_ver))
    contours = measure.find_contours(pad3, 0.5, fully_connected='low')

    contour_polygons = []
    cobter_feature_collection = []
    for n,cp in enumerate(contours):
        cp[:,1] = cp[:,1]-1
        cp[:,0] = cp[:,0]-1

        cp[np.where(cp[:,0]<0.),0] = 0
        cp[np.where(cp[:,0]>180.),0] = 180
        cp[np.where(cp[:,1]<0.),1] = 0
        cp[np.where(cp[:,1]>360.),1] = 360
        cpf = pygplates.PolygonOnSphere(zip(cp[:,0]-90,cp[:,1]-180))
        contour_polygons.append(cpf)
        cobter_feature_collection.append(cpf)
        
    area_threshold = 500000
    area_threshold2 = 10000

    tmp_mesoceno_area = []
    tmp_mesoceno_len = []
    tmp_mesoceno_area2 = []
    tmp_mesoceno_len2 = []
    for pg in contour_polygons:
        if pg.get_area()*pygplates.Earth.mean_radius_in_kms**2 > area_threshold:
            tmp_mesoceno_area.append(pg.get_area()*pygplates.Earth.mean_radius_in_kms**2)
            tmp_mesoceno_len.append(pg.get_arc_length()*pygplates.Earth.mean_radius_in_kms)
        if pg.get_area()*pygplates.Earth.mean_radius_in_kms**2 > area_threshold2:
            tmp_mesoceno_area2.append(pg.get_area()*pygplates.Earth.mean_radius_in_kms**2)
            tmp_mesoceno_len2.append(pg.get_arc_length()*pygplates.Earth.mean_radius_in_kms)

    plate_area_mesoceno.append(tmp_mesoceno_area)
    plate_length_mesoceno.append(tmp_mesoceno_len)
    plate_area_mesoceno2.append(tmp_mesoceno_area2)
    plate_length_mesoceno2.append(tmp_mesoceno_len2)

In [ ]:
timerange = np.arange(0,1010,10)

In [ ]:
plate_area = plate_area_mesoceno + plate_area_palaeo + plate_area_DomPalaeo + plate_area_neop
plate_length = plate_length_mesoceno + plate_length_palaeo + plate_length_DomPalaeo + plate_length_neop
plate_area2 = plate_area_mesoceno2 + plate_area_palaeo2 + plate_area_DomPalaeo2 + plate_area_neop2
plate_length2 = plate_length_mesoceno2 + plate_length_palaeo2 + plate_length_DomPalaeo2 + plate_length_neop2

summed_timesteps_area = []
for n,i in enumerate(plate_area):
    summed_timesteps_area.append(sum(i))
    
summed_timesteps_length = []
for n,j in enumerate(plate_length):
    summed_timesteps_length.append(sum(j))
    
summed_timesteps_area2 = []
for n,i in enumerate(plate_area2):
    summed_timesteps_area2.append(sum(i))
    
summed_timesteps_length2 = []
for n,j in enumerate(plate_length2):
    summed_timesteps_length2.append(sum(j))

In [ ]:
#save as a dictionary and export to csv
import csv

perimeter_area_ratio = dict(summed_timesteps_area = summed_timesteps_area,
                            summed_timesteps_length = summed_timesteps_length,
                            summed_timesteps_area2 = summed_timesteps_area2,
                            summed_timesteps_length2 = summed_timesteps_length2)

my_dict = perimeter_area_ratio

with open('perimeter_area_ratio.csv', 'wb') as f:  # Just use 'w' mode in 3.x
    w = csv.DictWriter(f, my_dict.keys())
    w.writeheader()
    w.writerow(my_dict)

In [ ]:
ratio = []
for a,l in zip(summed_timesteps_area, summed_timesteps_length):
    ratio.append(l/a)
    
ratio2 = []
for a2,l2 in zip(summed_timesteps_area2, summed_timesteps_length2):
    ratio2.append(l2/a2)

In [ ]:
total_area = []
total_area2 = []
for a in summed_timesteps_area:
    total_area.append(a)
    
for a2 in summed_timesteps_area2:
    total_area2.append(a2)

In [ ]:
#to normalise?
norm = [float(i)/max(ratio) for i in ratio]
norm2 = [float(j)/max(ratio2) for j in ratio2]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(30,12),facecolor='w', edgecolor='k')
ax.plot(timerange,ratio, 'r', label='500000 km2 area threshold')
ax.plot(timerange,ratio2, 'b', label='10000 km2 area threshold')
ax.set_xlim(1000,0)
#ax.set_ylim(0, 14)
ax.tick_params(axis='x', labelsize=30)
ax.tick_params(axis='y', labelsize=30)
ax.set_xlabel('Time (Ma)', fontsize=40)
ax.set_ylabel('Area (km2)', fontsize=40)
ax.set_ylabel('Perimeter / Area (km-1)', fontsize=40)
ax.legend(loc='upper left',fontsize=32,bbox_to_anchor=(0,1))

## Check for existing output directory and create it if not found
if not os.path.exists('%soutput' % basedir_save):
    os.makedirs('output')
#
    ## Check for existing output file with same name and remove if found
if os.path.isfile(('%soutput/' + str('Peri:Area')) % basedir_save):
    os.remove(('%soutput/' + str('Peri:Area')) % basedir_save)

#plt.savefig('%soutput1/%s.eps' % (basedir_save, time), format='eps', bbox_inches='tight', transparent=True)
plt.savefig('%soutput/Peri:Area.pdf' % basedir_save, format='pdf', bbox_inches='tight', transparent=True)

### now determine number of continents and their areas

In [ ]:
max_area = []
no_conts = []
for i,j in enumerate(plate_area):
    max_area.append(max(j))
    no_conts.append(len(j))
fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(30,12),facecolor='w', edgecolor='k')
ax2 = ax1.twinx()
ax1.plot(timerange,max_area, 'r', label='area of largest continent')
ax2.plot(timerange,no_conts, 'b', label='number of continents')
ax1.set_xlim(1000,0)
ax2.set_ylim(0, 14)
ax1.tick_params(axis='x', labelsize=30)
ax1.tick_params(axis='y', labelsize=30)
ax2.tick_params(axis='y', labelsize=30)
ax1.set_xlabel('Time (Ma)', fontsize=40)
ax1.set_ylabel('Area (km2)', fontsize=40)
ax2.set_ylabel('Count', fontsize=40)
ax1.legend(loc='upper left',fontsize=32,bbox_to_anchor=(0,.9))
ax2.legend(loc='upper left',fontsize=32,bbox_to_anchor=(0,1))
plt.savefig("Bradley_2011_metrics.pdf")